# Nueral RGB->D Paper Code Study

## 1. mGPU = True

When `mGPU=True`, call the function `warp_img_feats_mgpu()` and `_back_warp_homo_parallel_v1()`, defined in `~/mvs-depth/src/baselines/neuralrgbd/code/warping/homography.py`.

In [1]:
def warp_img_feats_mgpu(feat_img_src, d_candi, R,t, IntM_tensors, unit_ray_arrays_2D ):
    r'''
    Warp the feat_imgs_src to the reference view for all candidate depths
    Inputs:
        feat_img_src - list of source image features: each is NCHW format
        d_candi,
        R,t - list of rotation and transitions

    Output: 
    Feat_warped_src
    '''

    IntM_tensor = IntM_tensors.squeeze(0)
    P_ref_cuda = unit_ray_arrays_2D.squeeze(0)

    d_candi_cuda = torch.from_numpy(d_candi.astype(np.float32)).cuda()

    if isinstance(R, list) and isinstance(t, list):
        H, W, D = feat_img_src[0].shape[2], feat_img_src[0].shape[3], len(d_candi)
        Feat_warped_src = []
        for idx_view, feat_img_src_view in enumerate(feat_img_src):
            # Get term1 #
            term1 = IntM_tensor.matmul(t[idx_view]).unsqueeze(1)
            # Get term2 # 
            term2 = IntM_tensor.matmul(R[idx_view]).matmul(P_ref_cuda)
            
            feat_img_src_view_repeat = feat_img_src_view.repeat(len(d_candi), 1, 1, 1)

            feat_img_src_view_warp_par_d =  _back_warp_homo_parallel_v1(
                feat_img_src_view_repeat, 
                d_candi_cuda, 
                term1, 
                term2, 
                IntM_tensor, 
                H, 
                W) 

            Feat_warped_src.append(
                torch.transpose(feat_img_src_view_warp_par_d, dim0=0, dim1=1)
                )
    else: # added by CCJ;
        raise Exception("no implemented Error! R and t should be 'list' variables")
    """
    else:
        #NOTE:added by CCJ: this code seems not to be used !!!
        H, W, D = feat_img_src.shape[2], feat_img_src.shape[3], len(d_candi)
        feat_img_src_view = feat_img_src
        # Get term1 #
#        term1 = IntM_tensor.matmul(t).reshape(3,1)
        term1 = IntM_tensor.matmul(t).unsqueeze(1)
        # Get term2 #
        term2 = IntM_tensor.matmul(R).matmul(P_ref_cuda)
        
        feat_img_src_view_repeat = feat_img_src_view.repeat(len(d_candi), 1, 1, 1)
        feat_img_src_view_warp_par_d = \
                _back_warp_homo_parallel(feat_img_src_view_repeat,
                                         d_candi_cuda, term1, term2,
                                         cam_intrinsic, H, W)

        Feat_warped_src = torch.transpose(feat_img_src_view_warp_par_d, dim0=0, dim1=1 )
    """

In [4]:
def _back_warp_homo_parallel_v1(img_src, D, term1, term2, intrin_M, H, W, debug_inputs = None ):
    r'''
    Do the warpping for the src. view analytically using homography, given the
    depth d for the reference view: 
    p_src ~ term1  + term2 * d

    inputs:
    term1, term2 - 3 x n_pix matrix 
    P_ref - The 2D matrix form for the unit_array for the camera
    D - candidate depths. A tensor array on GPU

    img_src_warpped - warpped src. image 
    '''
    n_d = len(D)
    term2_cp = term2.repeat(n_d, 1, 1)

    P_src = term1.unsqueeze(0) + term2_cp * D.reshape(n_d,1,1)
    P_src = P_src / (P_src[:, 2, :].unsqueeze(1)  + 1e-10 ) 

    src_coords = torch.FloatTensor(n_d, H, W, 2).cuda()

    src_coords[:,:,:,0] = P_src[:, 0, :].reshape(n_d, H, W)
    src_coords[:,:,:,1] = P_src[:, 1, :].reshape(n_d, H, W)
    u_center, v_center  = intrin_M[0,2], intrin_M[1,2]
    src_coords[:,:,:,0] = (src_coords[:,:,:,0] - u_center) / u_center
    src_coords[:,:,:,1] = (src_coords[:,:,:,1] - v_center) / v_center 
    img_src_warpped = F.grid_sample(img_src, src_coords, mode='bilinear', padding_mode='zeros', align_corners=True) 
    return img_src_warpped

## 2. mGPU = False

When `mGPU=False`, call the function `warp_img_feats_v3()` and `_back_warp_homo_parallel()`, defined in `~/mvs-depth/src/baselines/neuralrgbd/code/warping/homography.py`.

In [2]:
def warp_img_feats_v3(feat_img_src, d_candi, R, t, cam_intrinsic):
    r'''
    Warp the feat_imgs_src to the reference view for all candidate depths
    Inputs:
        feat_img_src - list of source image features: each is NCHW format
        d_candi,
        R,t - list of rotation and transitions
        cam_intrinsic 

    Output: 
    Feat_warped_src
    '''

    IntM_tensor = cam_intrinsic['intrinsic_M_cuda'].cuda() # intrinsic matrix 3x3 on GPU
    P_ref_cuda = cam_intrinsic['unit_ray_array_2D'].cuda() # unit ray array in matrix form on GPU
    d_candi_cuda = torch.from_numpy(d_candi.astype(np.float32)).cuda()

    if isinstance(R, list) and isinstance(t, list):
        H, W, D = feat_img_src[0].shape[2], feat_img_src[0].shape[3], len(d_candi)
        Feat_warped_src = []
        for idx_view, feat_img_src_view in enumerate(feat_img_src):
            # Get term1 #
            term1 = IntM_tensor.matmul(t[idx_view]).unsqueeze(1)
            # Get term2 # 
            term2 = IntM_tensor.matmul(R[idx_view]).matmul(P_ref_cuda) 
            feat_img_src_view_repeat = feat_img_src_view.repeat(len(d_candi), 1, 1, 1) 
            feat_img_src_view_warp_par_d =  _back_warp_homo_parallel(
                feat_img_src_view_repeat, 
                d_candi_cuda, 
                term1, term2, 
                cam_intrinsic, H, W)
            Feat_warped_src.append( torch.transpose(feat_img_src_view_warp_par_d, dim0=0, dim1=1 ))

    else:
        H, W, D = feat_img_src.shape[2], feat_img_src.shape[3], len(d_candi)
        feat_img_src_view = feat_img_src

        # Get term1 #
        term1 = IntM_tensor.matmul(t).unsqueeze(1)

        # Get term2 #
        term2 = IntM_tensor.matmul(R).matmul(P_ref_cuda)
        
        feat_img_src_view_repeat = feat_img_src_view.repeat(len(d_candi), 1, 1, 1)
        feat_img_src_view_warp_par_d = _back_warp_homo_parallel(feat_img_src_view_repeat,
                                                                d_candi_cuda, term1, term2,
                                                                cam_intrinsic, H, W)

        Feat_warped_src = torch.transpose(feat_img_src_view_warp_par_d, dim0=0, dim1=1 )

    return Feat_warped_src

In [3]:
def _back_warp_homo_parallel(img_src, D, term1, term2, cam_intrinsics, H, W, debug_inputs = None ):
    r'''
    Do the warpping for the src. view analytically using homography, given the
    depth d for the reference view: 
    p_src ~ term1  + term2 * d

    inputs:
    term1, term2 - 3 x n_pix matrix 
    P_ref - The 2D matrix form for the unit_array for the camera
    D - candidate depths. A tensor array on GPU

    img_src_warpped - warpped src. image 
    '''
    n_d = len(D)
    term2_cp = term2.repeat(n_d, 1, 1)

    P_src = term1.unsqueeze(0) + term2_cp * D.reshape(n_d,1,1)
    P_src = P_src / (P_src[:, 2, :].unsqueeze(1)  + 1e-10 ) 

    src_coords = torch.FloatTensor(n_d, H, W, 2).cuda()

    src_coords[:,:,:,0] = P_src[:, 0, :].reshape(n_d, H, W)
    src_coords[:,:,:,1] = P_src[:, 1, :].reshape(n_d, H, W)
    u_center, v_center = cam_intrinsics['intrinsic_M'][0,2], cam_intrinsics['intrinsic_M'][1,2]
    src_coords[:,:,:,0] = (src_coords[:,:,:,0] - u_center) / u_center
    src_coords[:,:,:,1] = (src_coords[:,:,:,1] - v_center) / v_center 
    img_src_warpped = F.grid_sample(img_src, src_coords, mode='bilinear', padding_mode='zeros', align_corners=True) 
    return img_src_warpped